In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
os.chdir("..")

from scripts.ks_flat import ks93, ks93inv
from scripts.wf_flat import compute_s_from_theory, spin_wiener_filter

### This script uses a [Wiener filter (WF)](https://arxiv.org/pdf/1109.0932) for shear to convergence reconstruction:


WF uses the expected convergence field power spectrum (at a fiducial cosmology) and noise power spectrum. In Fourier space,


$ \tilde{\kappa} = \mathbf{SP}^\dagger \left[ \mathbf{PSP}^\dagger + \mathbf{N} \right]^{-1} \tilde{\gamma} $,


where $\textbf{S}$ is the covariance matrix of power spectrum signal, $\textbf{N}$ is the noise covariance matrix, and $\textbf{P}$ is the forward model:

$ \textbf{\textrm{P}} = \left( \frac{k_1^2 - k_2^2}{k^2} + i\frac{2k_1 k_2}{k^2} \right), $

where $k_1$ and $k_2$ are spatial frequencies.

<br>

##### 1. Generate shear maps

In [2]:
# get shear maps - start with a random covergence-like field (to be replaced by HACC simulation convergence maps)
np.random.seed(0)
nside = 1024
kE = np.random.rand(nside, nside) / 10
kB = np.zeros_like(kE) # set B mode kappa map to zeros
data_q, data_u = ks93inv(kE, kB)

##### 2. Generate noise covariance matrix

In [3]:
# simulation box size info
box_width_deg = 15  ## TO UPDATE WITH HACC INFO
box_width_pix = 256 ## TO UPDATE WITH HACC INFO
box_width_arcmin = box_width_deg * 60 # arcmin
pixel_size_arcmin = box_width_arcmin / box_width_pix # arcmin/pixel
pixel_size_deg = pixel_size_arcmin / 60 # degree/pixel
box_size_rad = np.radians(box_width_deg) # radians
pixel_size_rad = np.radians(pixel_size_deg) # radians/pixel

# galaxy shape noise level in galaxies/pixels
sig = 0.26 / np.sqrt(2.5 * pixel_size_arcmin**2) # units of galaxies per pixel

# set noise covariance matrix
ncov_diag_Q = np.tile(sig ** 2, (box_width_pix, box_width_pix))
ncov_diag_U = np.tile(sig ** 2, (box_width_pix, box_width_pix))

#### 4. Generate fiducial cosmology signal power spectrum

In [ ]:
# define fiducial cosmology - these values come from Table 2 in https://arxiv.org/pdf/2209.04662 (TO UPDATE WITH HACC INFO)
Omega_m = 0.26
sigma8 = 0.84
w0 = -1.0
n_s = 0.9649
Omega_b = 0.0493
h = 0.673

# list of cosmological parameters
cosmo_params = [Omega_m, sigma8, w0, n_s, Omega_b, h]

# Load 1D power P(k) for theory E-mode signal power spectrum
z = # INSERT HACC REDSHIFT VALUES
nz = # INSERT HACC REDSHIFT DISTRIBUTION VALUES
input_ps_map_E = compute_s_from_theory(cosmo_params, z, nz, box_width_pix, box_size_rad, pixel_size_rad)
input_ps_map_B = np.zeros_like(input_ps_map_E)

##### 5. Run WF reconstruction

* Note: The same fiducial power spectrum can be used for non-fiducial convergence maps

In [ ]:
wf_jax = spin_wiener_filter(data_q, data_u, ncov_diag_Q, ncov_diag_U, input_ps_map_E, input_ps_map_B)
kE_wf, kB_wf = ks93(*wf_jax)